<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2024/images/logo.png?raw=true" alt="2024年度ゲノム情報解析入門" height="100px" align="middle">

<div align="right"><a href="https://github.com/CropEvol/lecture#section2">実習表ページに戻る</a></div>

[課題] ゲノムワイド関連解析(GWAS)
---

　課題の回答は、PandAの回答フォームにご記入ください。

PandA課題ページ: https://panda.ecs.kyoto-u.ac.jp/x/h24tsn

　課題をはじめる前に、次のコードセルを一度実行してください。課題に必要なパッケージのインストールや、プログラム、サンプルファイルをダウンロードします。

In [ ]:
################################
##  以下の実習の前に一度実行してください。##
################################
## Rパッケージのインストール、プログラム・サンプルファイルのダウンロード
system("wget -q -O library.tar.gz https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2023/dataset/library.tar.gz")
system("wget -q -O GWAS_sample_genotype_hw.csv https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2023/dataset/GWAS_sample_genotype_hw.csv")
system("wget -q -O GWAS_sample_phenotype_hw.csv https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2023/dataset/GWAS_sample_phenotype_hw.csv")
untar("library.tar.gz")
.libPaths("library")
# 確認
library(rrBLUP)
library(ggplot2)


## データの読み込み
課題で扱うデータも、実習で扱った「ひとめぼれ系統」に様々なイネの系統を掛け合わせた集団になります。

ただし、今回扱う形質データは、３地点で計測した籾数のデータになります。

(同じ集団を青森、岩手、福島で栽培し、それぞれの地点で形質データを測定した)

In [ ]:
# 遺伝子型情報読み込み
rice_genotype <- read.csv("GWAS_sample_genotype_hw.csv")

In [ ]:
# 形質値データ読み込み&最初の数サンプルのみ表示
rice_phenotype <- read.csv("GWAS_sample_phenotype_hw.csv")
head(rice_phenotype)

## 課題1

以下の２つのコードを動かすと、岩手の籾数データでGWASを行った結果が表示されます。(実習でやったものと同じ)

その結果、籾数と関係してそうな領域(FDR<0.01)を３番染色体上に検出することが出来ます。

しかし、検出された領域には遺伝子が数百個位置しており、どの遺伝子が籾数に影響を与えるのかを決めるにはまだまだ難しい状況です。

**ここから候補領域をより狭い領域に絞っていったり、原因遺伝子を特定していきたい場合、更にどのようなアプローチを適用すると良いと考えられるでしょうか？**

自分の考えを課題1の回答として記入してください。

In [ ]:
# GWAS
library(rrBLUP)
X <- rice_genotype[, 4:dim(rice_genotype)[2]]
M <- X-1
geno <- data.frame(marker=rice_genotype$Marker, chrom=rice_genotype$Chrom, pos=rice_genotype$Position, M, check.names=FALSE)
GWAS_rice <- GWAS(rice_phenotype[,c(1,2)], geno, plot=FALSE)

# calculate FDR threshold
q_values_GN <- p.adjust(10 ** (-GWAS_rice$Iwate), method = "BH")
FDR_threshold_GN <- min(GWAS_rice$Iwate[q_values_GN < 0.01])

# visualize result
g <- ggplot(GWAS_rice, aes(x = marker, y = Iwate, color=chrom))
g <- g + geom_point()
g <- g + geom_hline(yintercept = FDR_threshold_GN, linetype = "dashed")
g <- g + ggtitle("Grain number in Iwate")
g <- g + ylim(c(0, 12))
g <- g + theme(text = element_text(size = 24))
plot(g)

In [ ]:
# show candidate region
candidate <- GWAS_rice[q_values_GN < 0.01, ]
print(paste(candidate$chrom[1], min(candidate$pos), "~", max(candidate$pos), "bp が有意な領域として検出された"))

## 課題2

以下のコードを動かすと、

岩手で栽培した集団を、３地点(青森、岩手、福島)それぞれで同じ様に栽培して得られた形質データを用いたGWASの結果が表示されます。

* 青森での栽培結果を用いたGWAS
* 岩手での栽培結果を用いたGWAS(課題1と同じ)
* 福島での栽培結果を用いたGWAS

同じ集団を育ててGWASを行っているので、同じようなGWASの結果になると予想されますが、

実際には場所毎に異なったGWASの結果が得られます。

**ではこの結果はどのように解釈することが出来るでしょうか？**

自分の考えを課題2の回答として記入してください。

In [ ]:
# draw manhattan plot for Grain number
GWAS_rice_3loc <- GWAS(rice_phenotype, geno, plot=FALSE)
for (place in c("Aomori", "Iwate", "Fukushima")) {
    q_values_GN <- p.adjust(10 ** (-GWAS_rice_3loc[,place]), method = "BH")
    FDR_threshold_GN <- min(GWAS_rice_3loc[,place][q_values_GN < 0.01])
    g <- ggplot(GWAS_rice_3loc, aes_(x = as.name("marker"), y = as.name(place), color=as.name("chrom")))
    g <- g + geom_point()
    g <- g + geom_hline(yintercept = FDR_threshold_GN, linetype = "dashed")
    g <- g + ggtitle(paste("Grain number in", place))
    g <- g + ylim(c(0, 12))
    g <- g + theme(text = element_text(size = 24))
    plot(g)
}

# 課題の提出について

提出期限： 2024年11月25日（月） まで

<div align="right"><a href="https://github.com/CropEvol/lecture#section2">実習表ページに戻る</a></div>